In [1]:
import os
os.chdir("/Users/jacksonwalters/Documents/GitHub/enefit-kaggle/predict-energy-behavior-of-prosumers/")
!ls

client.csv                            gas_prices.csv
county_id_to_name_map.json            historical_weather.csv
electricity_prices.csv                public_timeseries_testing_util.py
enefit                                train.csv
example_test_files                    weather_station_to_county_mapping.csv
forecast_weather.csv


In [2]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from load_data import merged_df

In [5]:
df = merged_df()

In [11]:
#https://www.kaggle.com/competitions/predict-energy-behavior-of-prosumers/discussion/455100

In [6]:
#drop the target column and data_block_id as they are not present in test data
X = df.drop(['target'], axis=1).rename(columns={'datetime': 'prediction_datetime'})

In [7]:
X.columns

Index(['county', 'is_business', 'product_type', 'is_consumption',
       'prediction_datetime', 'data_block_id', 'row_id', 'prediction_unit_id',
       'forecast_date', 'lowest_price_per_mwh', 'highest_price_per_mwh',
       'euros_per_mwh', 'latitude', 'longitude', 'hours_ahead', 'temperature',
       'dewpoint', 'cloudcover_high', 'cloudcover_low', 'cloudcover_mid',
       'cloudcover_total', '10_metre_u_wind_component',
       '10_metre_v_wind_component', 'direct_solar_radiation',
       'surface_solar_radiation_downwards', 'snowfall', 'total_precipitation'],
      dtype='object')

In [8]:
#set the target variable
y = df['target']

In [9]:
#verify there are no NaN's
np.any(np.isnan(y))

False

In [10]:
# load the modeling modules from sklearn
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import accuracy_score, confusion_matrix, precision_score, recall_score, ConfusionMatrixDisplay
from sklearn.model_selection import RandomizedSearchCV, train_test_split
from scipy.stats import randint
from sklearn.metrics import mean_absolute_error

In [11]:
# Split the data into training and test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)

In [12]:
#create a multivariable linear regression
from sklearn import linear_model
regr = linear_model.LinearRegression()
regr.fit(X_train, y_train)
regr.feature_names = list(X_train.columns.values)

In [13]:
#predict on the test data
lin_pred = regr.predict(X_test)

In [14]:
mean_absolute_error(lin_pred,y_test)

287.27452496188425

In [15]:
#train the random forest regressor
rf = RandomForestRegressor(n_estimators=1)
rf.fit(X_train, y_train)
rf.feature_names = list(X_train.columns.values)

In [16]:
#predict the target values using the random forest regressor
rf_pred = rf.predict(X_test)

In [17]:
#mean absolute error for random regressor
mean_absolute_error(y_test,rf_pred)

0.21447837559367064

In [34]:
#perform k-fold cross-validation
from sklearn.model_selection import KFold
kf = KFold(n_splits=5)

for train_index, test_index in kf.split(X):
    X_train, X_test = X.iloc[train_index], X.iloc[test_index]
    y_train, y_test = y.iloc[train_index], y.iloc[test_index]
    
    #train the model
    lin_regr = linear_model.LinearRegression()
    lin_regr.fit(X_train, y_train)
    #predict on the test data
    lin_pred = regr.predict(X_test)
    #evaluate the model
    print(mean_absolute_error(lin_pred,y_test))

273.4827216079995
285.00468175480364
269.84207813612034
297.5972492682266
309.99922880975254


In [36]:
import pickle
# save the linear model to disk
linear_filename = '../models/linear_model.sav'
pickle.dump(regr, open(linear_filename, 'wb'))
random_forest_filename = '../models/random_forest_model.sav'
pickle.dump(rf, open(random_forest_filename, 'wb'))